In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib

# Hello world

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Версії пакетів</b></summary>

Код на цій сторінці був розроблений з використанням наступних вимог.
Ми рекомендуємо використовувати ці версії або новіші.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
Цей приклад складається з двох частин. Спочатку Вистворите просту квантову програму та запустите її на квантовому процесорі (QPU). Оскільки справжні квантові дослідження потребують набагато надійніших програм, у другому розділі ([Масштабування до великої кількості кубітів](#scale-to-large-numbers-of-qubits)) Вимасштабуєте просту програму до рівня корисності.
## Встановлення та автентифікація
1. Якщо Вище не встановили Qiskit, знайдіть інструкції в посібнику [Швидкий старт](/guides/quick-start).

    - Встановіть Qiskit Runtime для виконання завдань на квантовому обладнанні:

        ```bash
        pip install qiskit-ibm-runtime
        ```

    - Налаштуйте середовище для локального запуску Jupyter notebooks:

        ```bash
        pip install jupyter
        ```

2. Налаштуйте автентифікацію для доступу до квантового обладнання через безкоштовний [Відкритий План](/guides/plans-overview#open-plan).

    (Якщо Вам надіслали запрошення приєднатися до облікового запису, натомість виконайте [кроки для запрошених користувачів](/guides/cloud-setup-invited).)

    - Перейдіть на [IBM Quantum Platform](https://quantum.cloud.ibm.com/), щоб увійти або створити обліковий запис.
         > **Note:** Якщо Випідключаєтеся через проксі-сервер, Виповинні використовувати Qiskit Runtime v0.44.0 або пізнішу версію.
    - Згенеруйте свій API ключ (також званий *API токеном*) на [панелі керування](https://quantum.cloud.ibm.com/), потім скопіюйте його в безпечне місце.
    - Перейдіть на сторінку [Екземпляри](https://quantum.cloud.ibm.com/instances) і знайдіть екземпляр, який Вихочете використовувати. Наведіть курсор на його CRN і клацніть, щоб скопіювати його.

    - Збережіть свої облікові дані локально за допомогою цього коду:

        ```python
        from qiskit_ibm_runtime import QiskitRuntimeService

        QiskitRuntimeService.save_account(
        token="<your-api-key>", # Use the 44-character API_KEY you created and saved from the IBM Quantum Platform Home dashboard
        instance="<CRN>", # Optional
        )
        ```

3. Тепер Виможете використовувати цей код Python будь-коли, коли Вам потрібно автентифікуватися в Qiskit Runtime Service:
    ```python
        from qiskit_ibm_runtime import QiskitRuntimeService

        # Run every time you need the service
        service = QiskitRuntimeService()
    ```
> **Info:** Якщо Вивикористовуєте публічний комп'ютер або інше незахищене середовище, натомість виконайте [інструкції ручної автентифікації](/guides/cloud-setup-untrusted), щоб зберегти Ваші облікові дані в безпеці.
## Створення та запуск простої квантової програми
Чотири кроки для написання квантової програми з використанням шаблонів Qiskit:

1.  Відобразіть задачу у квантовий формат.

2.  Оптимізуйте схеми та оператори.

3.  Виконайте за допомогою функції квантового примітиву.

4.  Проаналізуйте результати.

### Крок 1. Відображення задачі у квантовий формат
У квантовій програмі *квантові схеми* — це нативний формат для представлення квантових інструкцій, а *оператори* представляють спостережувані величини, які потрібно виміряти. Під час створення схеми Визазвичай створюєте новий об'єкт [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class), а потім послідовно додаєте до нього інструкції.
Наступна комірка коду створює схему, яка генерує *стан Белла*, який є станом, в якому два кубіти повністю заплутані один з одним.

> **Note:** Qiskit SDK використовує нумерацію бітів LSb 0, де $n$-а цифра має значення $1 \ll n$ або $2^n$. Для отримання додаткової інформації дивіться тему [Порядок бітів у Qiskit SDK](/guides/bit-ordering).

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorOptions
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from matplotlib import pyplot as plt
# Uncomment the next line if you want to use a simulator:
# from qiskit_ibm_runtime.fake_provider import FakeBelemV2


# Create a new circuit with two qubits
qc = QuantumCircuit(2)

# Add a Hadamard gate to qubit 0
qc.h(0)

# Perform a controlled-X gate on qubit 1, controlled by qubit 0
qc.cx(0, 1)

# Return a drawing of the circuit using MatPlotLib ("mpl").
# These guides are written by using Jupyter notebooks, which
# display the output of the last line of each cell.
# If you're running this in a script, use `print(qc.draw())` to
# print a text drawing.
qc.draw("mpl")

<Image src="../docs/images/guides/hello-world/extracted-outputs/930ca3b6-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/930ca3b6-0.svg)

Дивіться [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class) у документації для всіх доступних операцій.
Під час створення квантових схем Витакож повинні враховувати, який тип даних Вихочете отримати після виконання. Qiskit надає два способи повернення даних: Виможете отримати розподіл ймовірностей для набору кубітів, які Вивирішите виміряти, або Виможете отримати очікуване значення спостережуваної величини. Підготуйте своє завдання для вимірювання Вашої схеми одним з цих двох способів за допомогою [примітивів Qiskit](/guides/get-started-with-primitives) (детально пояснюється в [Кроці 3](#step-3-execute-using-the-quantum-primitives)).

Цей приклад вимірює очікувані значення, використовуючи підмодуль `qiskit.quantum_info`, який визначається за допомогою операторів (математичних об'єктів, що використовуються для представлення дії або процесу, який змінює квантовий стан). Наступна комірка коду створює шість двокубітних операторів Паулі: `IZ`, `IX`, `ZI`, `XI`, `ZZ` та `XX`.

In [2]:
# Set up six different observables.

observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]

> **Note:** Тут щось на зразок оператора `ZZ` — це скорочення для тензорного добутку $Z\otimes Z$, що означає вимірювання Z на кубіті 1 та Z на кубіті 0 разом та отримання інформації про кореляцію між кубітом 1 і кубітом 0. Очікувані значення, подібні до цього, також зазвичай записуються як $\langle Z_1 Z_0 \rangle$.
> 
> Якщо стан заплутаний, то вимірювання $\langle Z_1 Z_0 \rangle$ повинно відрізнятися від вимірювання $\langle I_1 \otimes Z_0 \rangle \langle Z_1 \otimes I_0 \rangle$. Для конкретного заплутаного стану, створеного нашою схемою, описаною вище, вимірювання $\langle Z_1 Z_0 \rangle$ повинно бути 1, а вимірювання $\langle I_1 \otimes Z_0 \rangle \langle Z_1 \otimes I_0 \rangle$ повинно бути нуль.
<span id="optimize"></span>

### Крок 2. Оптимізація схем та операторів

Під час виконання схем на пристрої важливо оптимізувати набір інструкцій, які містить схема, і мінімізувати загальну глибину (приблизно кількість інструкцій) схеми. Це забезпечує отримання найкращих можливих результатів шляхом зменшення впливу помилок та шуму. Крім того, інструкції схеми повинні відповідати [архітектурі набору інструкцій (ISA)](/guides/transpile#instruction-set-architecture) пристрою backend і повинні враховувати базові вентилі пристрою та зв'язність кубітів.

Наступний код створює екземпляр реального пристрою для подання завдання та перетворює схему та спостережувані величини відповідно до ISA цього backend. Це вимагає, щоб Вивже [зберегли свої облікові дані](/guides/cloud-setup)

In [ ]:
service = QiskitRuntimeService()

backend = service.least_busy(simulator=False, operational=True)

# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

isa_circuit.draw("mpl", idle_wires=False)

<Image src="../docs/images/guides/hello-world/extracted-outputs/9a901271-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/9a901271-0.svg)

### Крок 3. Виконання за допомогою квантових примітивів
Квантові комп'ютери можуть виробляти випадкові результати, тому Визазвичай збираєте вибірку виходів, запускаючи схему багато разів. Ви можете оцінити значення спостережуваної величини, використовуючи клас `Estimator`. `Estimator` є одним з двох [примітивів](/guides/get-started-with-primitives); інший — це `Sampler`, який можна використовувати для отримання даних з квантового комп'ютера. Ці об'єкти мають метод `run()`, який виконує вибір схем, спостережуваних величин та параметрів (якщо застосовно), використовуючи [примітивний уніфікований блок (PUB).](/guides/primitives#sampler)

In [4]:
# Construct the Estimator instance.

estimator = Estimator(mode=backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 5000

mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

# One pub, with one circuit to run against five different observables.
job = estimator.run([(isa_circuit, mapped_observables)])

# Use the job ID to retrieve your job data later
print(f">>> Job ID: {job.job_id()}")

>>> Job ID: d5k96q4jt3vs73ds5tgg


After a job is submitted, you can wait until either the job is completed within your current python instance, or use the `job_id` to retrieve the data at a later time.  (See the [section on retrieving jobs](/docs/guides/monitor-job#retrieve-job-results-at-a-later-time) for details.)

After the job completes, examine its output through the job's `result()` attribute.

In [5]:
# This is the result of the entire submission.  You submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had six observables,
# so contains information on all six.
pub_result = job.result()[0]

In [6]:
# Check there are six observables.
# If not, edit the comments in the previous cell and update this test.
assert len(pub_result.data.evs) == 6

<Admonition type="note" title="Alternative: run the example using a simulator">
  When you run your quantum program on a real device, your workload must wait in a queue before it runs. To save time, you can instead use the following code to run this small workload on the [`fake_provider`](../api/qiskit-ibm-runtime/fake-provider) with the Qiskit Runtime local testing mode. Note that this is only possible for a small circuit. When you scale up in the next section, you will need to use a real device.

  ```python

  # Use the following code instead if you want to run on a simulator:

  from qiskit_ibm_runtime.fake_provider import FakeBelemV2
  backend = FakeBelemV2()
  estimator = Estimator(backend)

  # Convert to an ISA circuit and layout-mapped observables.

  pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
  isa_circuit = pm.run(qc)
  mapped_observables = [
      observable.apply_layout(isa_circuit.layout) for observable in observables
  ]

  job = estimator.run([(isa_circuit, mapped_observables)])
  result = job.result()

  # This is the result of the entire submission.  You submitted one Pub,
  # so this contains one inner result (and some metadata of its own).

  job_result = job.result()

  # This is the result from our single pub, which had five observables,
  # so contains information on all five.

  pub_result = job.result()[0]
  ```
</Admonition>

### Step 4. Analyze the results

The analyze step is typically where you might post-process your results using, for example, measurement error mitigation or zero noise extrapolation (ZNE). You might feed these results into another workflow for further analysis or prepare a plot of the key values and data. In general, this step is specific to your problem.  For this example, plot each of the expectation values that were measured for our circuit.

The expectation values and standard deviations for the observables you specified to Estimator are accessed through the job result's `PubResult.data.evs` and `PubResult.data.stds` attributes. To obtain the results from Sampler, use the `PubResult.data.meas.get_counts()` function, which will return a `dict` of measurements in the form of bitstrings as keys and counts as their corresponding values. For more information, see [Get started with Sampler.](/docs/guides/get-started-with-primitives#get-started-with-sampler)

In [ ]:
# Plot the result

values = pub_result.data.evs

errors = pub_result.data.stds

# plotting graph
plt.plot(observables_labels, values, "-o")
plt.xlabel("Observables")
plt.ylabel("Values")
plt.show()

<Image src="../docs/images/guides/hello-world/extracted-outputs/87143fcc-0.svg" alt="Output of the previous code cell" />

> **Note:** Коли Визапускаєте свою квантову програму на реальному пристрої, Ваше завдання повинно чекати в черзі перед запуском. Щоб заощадити час, Виможете натомість використовувати наступний код для запуску цього невеликого завдання на [`fake_provider`](../api/qiskit-ibm-runtime/fake-provider) з режимом локального тестування Qiskit Runtime. Зверніть увагу, що це можливо лише для невеликої схеми. Коли Вимасштабуватимете в наступному розділі, Вам потрібно буде використовувати реальний пристрій.
> 
>   ```python
> 
>   # Use the following code instead if you want to run on a simulator:
> 
>   from qiskit_ibm_runtime.fake_provider import FakeBelemV2
>   backend = FakeBelemV2()
>   estimator = Estimator(backend)
> 
>   # Convert to an ISA circuit and layout-mapped observables.
> 
>   pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
>   isa_circuit = pm.run(qc)
>   mapped_observables = [
>       observable.apply_layout(isa_circuit.layout) for observable in observables
>   ]
> 
>   job = estimator.run([(isa_circuit, mapped_observables)])
>   result = job.result()
> 
>   # This is the result of the entire submission.  You submitted one Pub,
>   # so this contains one inner result (and some metadata of its own).
> 
>   job_result = job.result()
> 
>   # This is the result from our single pub, which had five observables,
>   # so contains information on all five.
> 
>   pub_result = job.result()[0]
>   ```
### Крок 4. Аналіз результатів
Крок аналізу зазвичай є місцем, де Виможете постобробити свої результати, використовуючи, наприклад, пом'якшення помилок вимірювання або екстраполяцію нульового шуму (ZNE). Ви можете передати ці результати в інший робочий процес для подальшого аналізу або підготувати графік ключових значень і даних. Загалом, цей крок є специфічним для Вашої задачі. Для цього прикладу побудуйте графік кожного з очікуваних значень, які були виміряні для нашої схеми.

Очікувані значення та стандартні відхилення для спостережуваних величин, які Вивказали для Estimator, доступні через атрибути `PubResult.data.evs` та `PubResult.data.stds` результату завдання. Щоб отримати результати з Sampler, використовуйте функцію `PubResult.data.meas.get_counts()`, яка поверне `dict` вимірювань у вигляді бітрядків як ключів та кількості як відповідних значень. Для отримання додаткової інформації дивіться [Початок роботи з Sampler.](/guides/get-started-with-primitives#get-started-with-sampler)

In [8]:
# Make sure the results follow the claim from the previous markdown cell.
# This can happen when the device occasionally behaves strangely. If this cell
# fails, you may just need to run the notebook again.

_results = {obs: val for obs, val in zip(observables_labels, values)}
for _label in ["IZ", "IX", "ZI", "XI"]:
    assert abs(_results[_label]) < 0.2
for _label in ["XX", "ZZ"]:
    assert _results[_label] > 0.8

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/87143fcc-0.svg)

Зверніть увагу, що для кубітів 0 і 1 незалежні очікувані значення як X, так і Z дорівнюють 0, тоді як кореляції (`XX` і `ZZ`) дорівнюють 1. Це є ознакою квантової заплутаності.

In [ ]:
def get_qc_for_n_qubit_GHZ_state(n: int) -> QuantumCircuit:
    """This function will create a qiskit.QuantumCircuit (qc) for an n-qubit GHZ state.

    Args:
        n (int): Number of qubits in the n-qubit GHZ state

    Returns:
        QuantumCircuit: Quantum circuit that generate the n-qubit GHZ state, assuming all qubits start in the 0 state
    """
    if isinstance(n, int) and n >= 2:
        qc = QuantumCircuit(n)
        qc.h(0)
        for i in range(n - 1):
            qc.cx(i, i + 1)
    else:
        raise Exception("n is not a valid input")
    return qc


# Create a new circuit with two qubits (first argument) and two classical
# bits (second argument)
n = 100
qc = get_qc_for_n_qubit_GHZ_state(n)

## Масштабування до великої кількості кубітів
У квантових обчисленнях робота на рівні корисності має вирішальне значення для досягнення прогресу в цій галузі. Така робота вимагає обчислень у набагато більшому масштабі; роботи зі схемами, які можуть використовувати понад 100 кубітів і понад 1000 вентилів. Цей приклад демонструє, як Виможете виконувати роботу на рівні корисності на QPU IBM&reg;, створюючи та аналізуючи 100-кубітний стан GHZ. Він використовує робочий процес шаблонів Qiskit і закінчується вимірюванням очікуваного значення $\langle Z_0 Z_i \rangle $ для кожного кубіта.

### Крок 1. Відображення задачі
Напишіть функцію, яка повертає `QuantumCircuit`, що підготовлює $n$-кубітний стан GHZ (по суті, розширений стан Белла), потім використовуйте цю функцію для підготовки 100-кубітного стану GHZ і зберіть спостережувані величини для вимірювання.

In [ ]:
# ZZII...II, ZIZI...II, ... , ZIII...IZ
operator_strings = [
    "Z" + "I" * i + "Z" + "I" * (n - 2 - i) for i in range(n - 1)
]
print(operator_strings)
print(len(operator_strings))

operators = [SparsePauliOp(operator) for operator in operator_strings]

['ZZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

Далі відобразіть на оператори, які Вас цікавлять. Цей приклад використовує оператори `ZZ` між кубітами для вивчення поведінки в міру їх віддалення. Все більш неточні (пошкоджені) очікувані значення між віддаленими кубітами виявили б рівень присутнього шуму.

In [ ]:
service = QiskitRuntimeService()

backend = service.least_busy(
    simulator=False, operational=True, min_num_qubits=100
)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

isa_circuit = pm.run(qc)
isa_operators_list = [op.apply_layout(isa_circuit.layout) for op in operators]

### Step 3. Execute on hardware

Submit the job and enable error suppression by using a technique to reduce errors called [dynamical decoupling.](../api/qiskit-ibm-runtime/options-dynamical-decoupling-options) The resilience level specifies how much resilience to build against errors. Higher levels generate more accurate results, at the expense of longer processing times.  For further explanation of the options set in the following code, see [Configure error mitigation for Qiskit Runtime.](/docs/guides/configure-error-mitigation)

In [ ]:
options = EstimatorOptions()
options.resilience_level = 1
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = "XY4"

# Create an Estimator object
estimator = Estimator(backend, options=options)

In [13]:
# Submit the circuit to Estimator
job = estimator.run([(isa_circuit, isa_operators_list)])
job_id = job.job_id()
print(job_id)

d5k9mmqvcahs73a1ni3g


### Крок 3. Виконання на обладнанні
Подайте завдання та увімкніть придушення помилок, використовуючи техніку зменшення помилок під назвою [динамічне роз'єднання.](../api/qiskit-ibm-runtime/options-dynamical-decoupling-options) Рівень стійкості визначає, яку стійкість до помилок слід забезпечити. Вищі рівні генерують більш точні результати за рахунок більш тривалого часу обробки. Для подальшого пояснення опцій, встановлених у наступному коді, дивіться [Налаштування пом'якшення помилок для Qiskit Runtime.](/guides/configure-error-mitigation)

In [ ]:
# data
data = list(range(1, len(operators) + 1))  # Distance between the Z operators
result = job.result()[0]
values = result.data.evs  # Expectation value at each Z operator.
values = [
    v / values[0] for v in values
]  # Normalize the expectation values to evaluate how they decay with distance.

# plotting graph
plt.plot(data, values, marker="o", label="100-qubit GHZ state")
plt.xlabel("Distance between qubits $i$")
plt.ylabel(r"$\langle Z_i Z_0 \rangle / \langle Z_1 Z_0 \rangle $")
plt.legend()
plt.show()

<Image src="../docs/images/guides/hello-world/extracted-outputs/de91ebd0-0.svg" alt="Output of the previous code cell" />

The previous plot shows that as the distance between qubits increases, the signal decays because of the presence of noise.

## Next steps

<Admonition type="tip" title="Recommendations">
  -   Try one of these tutorials:
      - [Ground-state energy estimation of the Heisenberg chain with VQE](/docs/tutorials/spin-chain-vqe)
      - Solve optimization problems using [QAOA](/docs/tutorials/quantum-approximate-optimization-algorithm)
      - Train [quantum kernel](/docs/tutorials/quantum-kernel-training) models for machine learning tasks
  - Find detailed installation instructions in the [Install Qiskit](/docs/guides/install-qiskit) guide.
  - If you prefer not to install Qiskit locally, read about options to use Qiskit in an [online development environment.](/docs/guides/online-lab-environments)
  - To save multiple account credentials or to specify other account options, see detailed instructions in the [Save your login credentials](/docs/guides/save-credentials#save-your-access-credentials) guide.
</Admonition>